### load_summarize_chain

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI
from langchain.callbacks.base import BaseCallbackHandler

# 웹 기반 문서 로더를 초기화합니다.
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")

# 문서를 로드합니다.
docs = loader.load()


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs):
        print(f"{token}", end="", flush=True)


# OpenAI의 Chat 모델을 초기화합니다. 여기서는 온도를 0으로 설정하고 모델 이름을 지정합니다.
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo-16k",
    streaming=True,
    callbacks=[StreamCallback()],
)
# 요약 체인을 로드합니다. 체인 타입을 'stuff'로 지정합니다.
chain = load_summarize_chain(llm, chain_type="stuff")

# 문서에 대해 요약 체인을 실행합니다.
answer = chain.invoke({"input_documents": docs})
print(answer["output_text"])

USER_AGENT environment variable not set, consider setting it to identify your requests.


The article discusses the concept of building autonomous agents powered by large language models (LLMs). It explores the components of such agents, including planning, memory, and tool use. The article provides case studies and examples of proof-of-concept demos, highlighting the challenges and limitations of LLM-powered agents. It also includes citations and references for further reading.The article discusses the concept of building autonomous agents powered by large language models (LLMs). It explores the components of such agents, including planning, memory, and tool use. The article provides case studies and examples of proof-of-concept demos, highlighting the challenges and limitations of LLM-powered agents. It also includes citations and references for further reading.


### 방법1. stuff방법1. stuff

1. 먼저, PromptTemplate를 사용하여 요약문 작성을 위한 프롬프트를 정의합니다.
2. 그 다음, LLMChain을 사용하여 지정된 모델(gpt-3.5-turbo-16k)과 온도 설정(0)을 사용하는 언어 모델 체인을 생성합니다.
3. 이 체인은 입력된 텍스트에 대한 요약문을 생성하는 데 사용됩니다.
4. 마지막으로, StuffDocumentsChain을 사용하여 문서들을 결합하고, 이를 LLMChain을 통해 요약합니다.
5. 이 과정은 loader.load()로 로드된 문서들에 대해 실행되며, 결과는 실시간 출력됩니다.

In [3]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain import hub

# 요약문을 작성하기 위한 프롬프트 정의 (직접 프롬프트를 작성하는 경우)
# prompt_template = """Please summarize the sentence according to the following REQUEST.
# REQUEST:
# 1. Summarize the main points in bullet points in KOREAN.
# 2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.
# 3. Use various emojis to make the summary more interesting.
# 4. Translate the summary into Korean if it is written in English.
# 5. DO NOT translate any technical terms.
# 6. DO NOT include any unnecessary information.
# CONTEXT:
# {context}

# SUMMARY:"
# """
# prompt = PromptTemplate.from_template(prompt_template)

# 원격 저장소에서 프롬프트를 가져오는 경우
prompt = hub.pull("teddynote/summary-stuff-documents-korean")

# LLM 체인 정의
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo-16k",
    streaming=True,
    callbacks=[StreamCallback()],
)

# LLMChain 정의
llm_chain = LLMChain(llm=llm, prompt=prompt)

# StuffDocumentsChain 정의
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="context")

docs = loader.load()
response = stuff_chain.invoke({"input_documents": docs})

c:\Users\UserK\miniforge3\envs\rag\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


🤖 LLM을 사용한 자율 에이전트 시스템은 LLM을 에이전트의 뇌로 사용하고 계획, 메모리, 도구 사용과 같은 여러 구성 요소로 보완됩니다.
📝 계획 구성 요소는 큰 작업을 작은 하위 목표로 분해하고 에이전트가 과거 행동을 자가 비판하고 반영하여 최종 결과의 품질을 향상시킵니다.
🧠 메모리 구성 요소는 감각 메모리, 단기 메모리, 장기 메모리로 구성되며, 외부 벡터 저장소와 빠른 검색을 통해 정보를 보존하고 검색할 수 있습니다.
🛠️ 도구 사용 구성 요소는 외부 API를 호출하여 모델 가중치에 없는 추가 정보를 얻을 수 있습니다.
🔍 이러한 구성 요소를 사용하여 LLM을 기반으로 한 에이전트를 구축하는 것은 재미있는 개념이지만, 제한된 컨텍스트 길이, 계획 및 작업 분해의 어려움, 자연어 인터페이스의 신뢰성 등의 제한 사항이 있습니다.

### 방법2. Map-Reduce

1. ChatOpenAI 인스턴스를 생성하고, 이를 사용하여 문서 집합에 대한 주요 테마를 식별하는 맵(map) 작업을 정의합니다.
2. 맵 작업은 map_template을 사용하여 정의되며, 이 템플릿은 문서 집합을 입력으로 받아 주요 테마를 식별하도록 요청합니다.
3. PromptTemplate.from_template 메서드를 사용하여 map_template에서 프롬프트 템플릿을 생성하고, LLMChain을 사용하여 맵 작업을 실행합니다.

In [4]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.llm import LLMChain
from langchain import hub


llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo",
    streaming=True,
    callbacks=[StreamCallback()],
)

# # map-prompt 를 직접 정의하는 경우 다음의 예시를 참고하세요.
# map_template = """The following is a set of documents
# {docs}
# Based on this list of docs, please identify the main themes
# Helpful Answer:"""
# map_prompt = PromptTemplate.from_template(map_template)

# langchain 허브에서 'rlm/map-prompt'를 가져옵니다.
map_prompt = hub.pull("teddynote/map-prompt")
map_prompt

PromptTemplate(input_variables=['docs'], metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'map-prompt', 'lc_hub_commit_hash': '5325a713fc858810667d1d1dde32ccc2e93433b8706831560e75360b4993e95f'}, template='You are a helpful expert journalist in extracting the main themes from a GIVEN DOCUMENTS below.\nPlease provide a comprehensive summary of the GIVEN DOCUMENTS in numbered list format. \nThe summary should cover all the key points and main ideas presented in the original text, while also condensing the information into a concise and easy-to-understand format. \nPlease ensure that the summary includes relevant details and examples that support the main ideas, while avoiding any unnecessary information or repetition. \nThe length of the summary should be appropriate for the length and complexity of the original text, providing a clear and accurate overview without omitting any important information.\n\nGIVEN DOCUMENTS:\n{docs}\n\nFORMAT:\n1. main theme 1\n2. main theme 2\n3. main th

In [5]:
# LLMChain 인스턴스를 생성하며, 이때 LLM과 프롬프트로 'map_prompt'를 사용합니다.
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [6]:
# reduce-prompt 를 직접 정의하는 경우 다음의 예시를 참고하세요.
# reduce_template = """The following is set of summaries:
# {docs}
# Take these and distill it into a final, consolidated summary of the main themes.
# Helpful Answer:"""
# reduce_prompt = PromptTemplate.from_template(reduce_template)

In [7]:
# prompt hub에서도 얻을 수 있음을 위에서 언급했듯이
reduce_prompt = hub.pull("teddynote/reduce-prompt-korean")
reduce_prompt

PromptTemplate(input_variables=['doc_summaries'], metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'reduce-prompt-korean', 'lc_hub_commit_hash': '01613c7c2988c1e28d025507398b6c4aa4484e4450186e377b8e578bd22077ab'}, template='You are a helpful expert in summary writing.\nYou are given numbered lists of summaries.\nExtract top 10 most important insights from the summaries.\nThen, write a summary of the insights in KOREAN.\n\nLIST OF SUMMARIES:\n{doc_summaries}\n\nHelpful Answer:\n')

In [8]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# 연쇄 실행
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# 문서 리스트를 받아 하나의 문자열로 결합한 후 LLMChain에 전달
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# 매핑된 문서들을 결합하고 반복적으로 축소
reduce_documents_chain = ReduceDocumentsChain(
    # 최종적으로 호출되는 체인입니다.
    combine_documents_chain=combine_documents_chain,
    # `StuffDocumentsChain`의 컨텍스트를 초과하는 문서들을 처리
    collapse_documents_chain=combine_documents_chain,
    # 문서들을 그룹화할 최대 토큰 수.
    token_max=4096,
)

In [9]:
# 문서들을 매핑하여 체인을 거친 후 결과를 결합하는 과정
map_reduce_chain = MapReduceDocumentsChain(
    # 매핑 체인
    llm_chain=map_chain,
    # 리듀스 체인
    reduce_documents_chain=reduce_documents_chain,
    # llm_chain에서 문서들을 넣을 변수 이름
    document_variable_name="docs",
    # 매핑 단계의 결과를 출력에 포함시킴
    return_intermediate_steps=False,
)

# 문자를 기준으로 텍스트를 분할하는 객체 생성
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=len,
)

# 문서들을 분할
split_docs = text_splitter.split_documents(docs)

In [10]:
# split_docs를 map_reduce_chain의 run 메서드에 전달하여 실행한 결과를 출력합니다.
summary_result = map_reduce_chain.invoke({"input_documents": split_docs})

1. The document discusses LLM powered autonomous agents, focusing on three main components: planning, memory, and tool use. 
2. Under planning, the document covers task decomposition and self-reflection as key aspects of agent systems. 
3. Memory is explored through different types and the use of Maximum Inner Product Search (MIPS) for efficient retrieval. 
4. Tool use is exemplified through case studies such as the Scientific Discovery Agent and Generative Agents Simulation, along with proof-of-concept examples. 
5. The document also highlights challenges faced by autonomous agents and provides citations and references for further reading.1. LLM (large language model) can serve as the core controller for building autonomous agents, showcasing potential beyond generating content like stories and essays.
2. In an LLM-powered agent system, key components include planning, which involves breaking down tasks into subgoals for efficient handling, and memory, which allows for self-reflection

In [11]:
print(summary_result["output_text"])

기억 스트림, 상호 에이전트 통신, 검색 모델, 반사 메커니즘, 계획, 에이전트 간의 관계, 환경 정보는 에이전트의 행동을 안내하고 영향을 미치는 중요한 요소들입니다.


### 방법3. Refine

In [12]:
# llm을 사용하여 'refine' 유형의 요약 체인을 로드합니다.
chain = load_summarize_chain(llm, chain_type="refine")
# split_docs를 처리하기 위해 체인을 실행합니다.
chain.run(split_docs)

c:\Users\UserK\miniforge3\envs\rag\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


The article discusses LLM Powered Autonomous Agents, focusing on three main components: planning, memory, and tool use. It explores task decomposition, self-reflection, types of memory, and the use of tools in case studies such as scientific discovery and generative agents simulation. The article also addresses challenges and provides citations and references.The article discusses LLM Powered Autonomous Agents, focusing on three main components: planning, memory, and tool use. It explores task decomposition, self-reflection, types of memory, and the use of tools in case studies such as scientific discovery and generative agents simulation. The potential of LLM extends beyond generating well-written copies, stories, essays, and programs; it can be framed as a powerful general problem solver. The article also addresses challenges and provides citations and references.The article discusses LLM Powered Autonomous Agents, focusing on three main components: planning, memory, and tool use. It

'The GPT-Engineer project focuses on task decomposition and optimization, incorporating user input for clarification. It explores various applications and techniques for autonomous agents powered by LLM. To proceed, a detailed implementation plan is necessary, defining core classes, functions, and methods, and writing structured code for each component. Python is the preferred tool, with tools like pytest and dataclasses for testing and data management. The project involves enhancing user experience by incorporating conversation samples. Challenges may arise in addressing limitations of LLM-centered agents, such as long-term planning difficulties. The project draws inspiration from recent research on large language models to improve agent capabilities. Additional related research includes topics like chemistry tools augmentation and emergent autonomous scientific research capabilities of large language models. The project also delves into prompt engineering and explores adversarial att

In [ ]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in Korean"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain.invoke({"input_documents": split_docs}, return_only_outputs=True)